In [1]:
import os
import copy
import torch

import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.nn.utils.prune as prune
import torchvision.transforms as transforms

from eval import accuracy
from model import MnistCNN
from train import fit, evaluate
from torchvision.datasets import MNIST
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
%matplotlib inline

In [2]:
BATCH_SIZE = 100
NUM_CLASSES = 10
INPUT_SIZE = 28 * 28
DATASET = MNIST(root='data/', download=True, train=True, transform=transforms.ToTensor())

In [3]:
if torch.cuda.is_available():  
    dev = "cuda:0" 
else:  
    dev = "cpu" 

def split_indices(n, val_pct):
    # Determine size of validation set
    n_val = int(val_pct*n)

    idxs = np.random.permutation(n)

    return idxs[n_val:], idxs[:n_val]

train_indexes, validation_indexes = split_indices(len(DATASET), 0.2)

train_sampler = SubsetRandomSampler(train_indexes)
train_loader = DataLoader(DATASET, BATCH_SIZE, sampler=train_sampler)

val_sampler = SubsetRandomSampler(validation_indexes)
val_loader = DataLoader(DATASET, BATCH_SIZE, sampler=val_sampler)

loss_fun = F.cross_entropy

def predict_image(image, model):
    xb = image.unsqueeze(0).to(dev)
    yb = model(xb)
    yb = yb.to(dev)
    _, preds = torch.max(yb, dim=1)
    return preds[0].item()

def plot_graph(rng ,acc_list, lost_list, title):
    plt.plot(rng, acc_list, '-b', label='accuracy')
    plt.plot(rng, lost_list, '-r', label='loss')
    plt.legend(loc='upper left')
    plt.xlabel("amount pruned")
    plt.title(title)
    plt.show()


In [4]:
learning_rate = 0.001

model = MnistCNN().to(dev)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
fit(10, model, loss_fun, optimizer, train_loader, val_loader, accuracy)

torch.save(model.state_dict(), 'model.pt')
# else:
#     model = MnistCNN()
#     model.load_state_dict(torch.load('model.pt', map_location=torch.device(dev)))
#     model.to(dev)

Epoch 1, loss: 0.0985, accuracy: 0.9705
Epoch 2, loss: 0.0760, accuracy: 0.9758
Epoch 3, loss: 0.0624, accuracy: 0.9818
Epoch 4, loss: 0.0655, accuracy: 0.9808
Epoch 5, loss: 0.0714, accuracy: 0.9798
Epoch 6, loss: 0.0629, accuracy: 0.9835
Epoch 7, loss: 0.0634, accuracy: 0.9832
Epoch 8, loss: 0.0774, accuracy: 0.9803
Epoch 9, loss: 0.0577, accuracy: 0.9845
Epoch 10, loss: 0.0623, accuracy: 0.9842


In [5]:
test_dataset = MNIST(root='data/', train=False, transform=transforms.ToTensor())
test_loader = DataLoader(test_dataset, batch_size=200)
test_loss, total, test_acc = evaluate(model, loss_fun, test_loader, metric=accuracy)
print(f"Loss: {test_loss:.4f}, accuracy: {test_acc:.4f}")

Loss: 0.0569, accuracy: 0.9846


In [6]:
def measure_module_sparsity(module, weight=True, bias=False, use_mask=False):

    num_zeros = 0
    num_elements = 0

    if use_mask == True:
        for buffer_name, buffer in module.named_buffers():
            if "weight_mask" in buffer_name and weight == True:
                num_zeros += torch.sum(buffer == 0).item()
                num_elements += buffer.nelement()
            if "bias_mask" in buffer_name and bias == True:
                num_zeros += torch.sum(buffer == 0).item()
                num_elements += buffer.nelement()
    else:
        for param_name, param in module.named_parameters():
            if "weight" in param_name and weight == True:
                num_zeros += torch.sum(param == 0).item()
                num_elements += param.nelement()
            if "bias" in param_name and bias == True:
                num_zeros += torch.sum(param == 0).item()
                num_elements += param.nelement()

    sparsity = num_zeros / num_elements

    return num_zeros, num_elements, sparsity

def measure_global_sparsity(model,
                            weight=True,
                            bias=False,
                            conv2d_use_mask=False,
                            linear_use_mask=False):

    num_zeros = 0
    num_elements = 0

    for module_name, module in model.named_modules():

        if isinstance(module, torch.nn.Conv2d):

            module_num_zeros, module_num_elements, _ = measure_module_sparsity(
                module, weight=weight, bias=bias, use_mask=conv2d_use_mask)
            num_zeros += module_num_zeros
            num_elements += module_num_elements

        elif isinstance(module, torch.nn.Linear):

            module_num_zeros, module_num_elements, _ = measure_module_sparsity(
                module, weight=weight, bias=bias, use_mask=linear_use_mask)
            num_zeros += module_num_zeros
            num_elements += module_num_elements

    sparsity = num_zeros / num_elements

    return num_zeros, num_elements, sparsity

In [7]:
for i in np.arange(0.1, 0.6, 0.1):
    prune_model = MnistCNN()
    prune_model.load_state_dict(copy.deepcopy(model.state_dict()))
    optimizer = torch.optim.Adam(prune_model.parameters(), lr=learning_rate)

    parameters_to_prune = (
        (prune_model.conv1, 'weight'),
        (prune_model.conv2, 'weight'),
        (prune_model.out, 'weight')
    )

    prune.global_unstructured(
        parameters_to_prune,
        pruning_method=prune.L1Unstructured,
        amount=i,
    )

    test_loss, total, test_acc = evaluate(prune_model.to(dev), loss_fun, test_loader, metric=accuracy)
    print(f"Amount pruned: {i}, Loss: {test_loss:.4f}, accuracy: {test_acc:.4f}")
    num_zeros, num_elements, sparsity = measure_global_sparsity(prune_model, True, False, True, False)
    print(f'sparsity: {sparsity}.')

    # Perform finetuning of model
    fit(5, prune_model, loss_fun, optimizer, train_loader, val_loader, accuracy) 
    test_loss, total, test_acc = evaluate(prune_model, loss_fun, test_loader, metric=accuracy)
    print(f"Loss: {test_loss:.4f}, accuracy: {test_acc:.4f}")
    print()

Amount pruned: 0.1, Loss: 0.0571, accuracy: 0.9847
sparsity: 0.09338642659279778.
Epoch 1, loss: 0.0676, accuracy: 0.9836
Epoch 2, loss: 0.0664, accuracy: 0.9857
Epoch 3, loss: 0.0794, accuracy: 0.9827
Epoch 4, loss: 0.0739, accuracy: 0.9844
Epoch 5, loss: 0.0652, accuracy: 0.9861
Loss: 0.0519, accuracy: 0.9863

Amount pruned: 0.2, Loss: 0.0583, accuracy: 0.9840
sparsity: 0.18708448753462603.
Epoch 1, loss: 0.0621, accuracy: 0.9856
Epoch 2, loss: 0.0651, accuracy: 0.9850
Epoch 3, loss: 0.0655, accuracy: 0.9857
Epoch 4, loss: 0.0651, accuracy: 0.9855
Epoch 5, loss: 0.0731, accuracy: 0.9848
Loss: 0.0550, accuracy: 0.9863

Amount pruned: 0.30000000000000004, Loss: 0.0730, accuracy: 0.9784
sparsity: 0.2789819944598338.
Epoch 1, loss: 0.0614, accuracy: 0.9846
Epoch 2, loss: 0.0712, accuracy: 0.9837
Epoch 3, loss: 0.0693, accuracy: 0.9842
Epoch 4, loss: 0.0723, accuracy: 0.9842
Epoch 5, loss: 0.0697, accuracy: 0.9848
Loss: 0.0538, accuracy: 0.9853

Amount pruned: 0.4, Loss: 0.2171, accuracy:

In [8]:
for i in np.arange(0.1, 0.6, 0.1):
    prune_model = MnistCNN()
    prune_model.load_state_dict(copy.deepcopy(model.state_dict()))
    optimizer = torch.optim.Adam(prune_model.parameters(), lr=learning_rate)

    parameters_to_prune = (
        (prune_model.conv1, 'weight'),
        (prune_model.conv2, 'weight'),
        (prune_model.out, 'weight')
    )

    prune.global_unstructured(
        parameters_to_prune,
        pruning_method=prune.RandomUnstructured,
        amount=i,
    )

    test_loss, total, test_acc = evaluate(prune_model.to(dev), loss_fun, test_loader, metric=accuracy)
    print(f"Amount pruned: {i}, Loss: {test_loss:.4f}, accuracy: {test_acc:.4f}")
    num_zeros, num_elements, sparsity = measure_global_sparsity(prune_model, True, False, True, False)
    print(f'sparsity: {sparsity}.')

    # Perform finetuning of model
    fit(5, prune_model, loss_fun, optimizer, train_loader, val_loader, accuracy) 
    test_loss, total, test_acc = evaluate(prune_model, loss_fun, test_loader, metric=accuracy)
    print(f"Loss: {test_loss:.4f}, accuracy: {test_acc:.4f}")
    print()

Amount pruned: 0.1, Loss: 0.3008, accuracy: 0.9084
sparsity: 0.046814404432132965.
Epoch 1, loss: 0.0799, accuracy: 0.9775
Epoch 2, loss: 0.0631, accuracy: 0.9827
Epoch 3, loss: 0.0575, accuracy: 0.9857
Epoch 4, loss: 0.0580, accuracy: 0.9846
Epoch 5, loss: 0.0599, accuracy: 0.9848
Loss: 0.0546, accuracy: 0.9856

Amount pruned: 0.2, Loss: 1.0081, accuracy: 0.7278
sparsity: 0.08999307479224376.
Epoch 1, loss: 0.0647, accuracy: 0.9818
Epoch 2, loss: 0.0629, accuracy: 0.9832
Epoch 3, loss: 0.0565, accuracy: 0.9856
Epoch 4, loss: 0.0580, accuracy: 0.9849
Epoch 5, loss: 0.0579, accuracy: 0.9848
Loss: 0.0455, accuracy: 0.9860

Amount pruned: 0.30000000000000004, Loss: 0.7512, accuracy: 0.7570
sparsity: 0.13639196675900278.
Epoch 1, loss: 0.0688, accuracy: 0.9790
Epoch 2, loss: 0.0564, accuracy: 0.9831
Epoch 3, loss: 0.0630, accuracy: 0.9811
Epoch 4, loss: 0.0549, accuracy: 0.9838
Epoch 5, loss: 0.0577, accuracy: 0.9839
Loss: 0.0483, accuracy: 0.9848

Amount pruned: 0.4, Loss: 1.8453, accurac

In [12]:
for i in np.arange(0.1, 0.6, 0.1):
    prune_model = MnistCNN()
    prune_model.load_state_dict(copy.deepcopy(model.state_dict()))
    optimizer = torch.optim.Adam(prune_model.parameters(), lr=learning_rate)

    parameters_to_prune = (
        (prune_model.conv1, 'weight'),
        (prune_model.conv2, 'weight'),
        (prune_model.out, 'weight')
    )

    prune.ln_structured(
        parameters_to_prune,
        pruning_method=prune.L1Unstructured,
        amount=i,
    )

    test_loss, total, test_acc = evaluate(prune_model.to(dev), loss_fun, test_loader, metric=accuracy)
    print(f"Amount pruned: {i}, Loss: {test_loss:.4f}, accuracy: {test_acc:.4f}")
    num_zeros, num_elements, sparsity = measure_global_sparsity(prune_model, True, False, True, False)
    print(f'sparsity: {sparsity}.')

    # Perform finetuning of model
    fit(5, prune_model, loss_fun, optimizer, train_loader, val_loader, accuracy) 
    test_loss, total, test_acc = evaluate(prune_model, loss_fun, test_loader, metric=accuracy)
    print(f"Loss: {test_loss:.4f}, accuracy: {test_acc:.4f}")
    print()

TypeError: ln_structured() got an unexpected keyword argument 'pruning_method'